In [1]:
'''更新日志10July
之前的原始牙齿体素模型除了牙齿外还包含很大一部分口腔，这一版里更改了原始牙齿文件，只有牙体。
简单目测，结果误差略有缩小，至少模型没变坏。
'''

'更新日志10July\n之前的原始牙齿体素模型除了牙齿外还包含很大一部分口腔，这一版里更改了原始牙齿文件，只有牙体。\n'

In [2]:
'''更新日志09July
这是第一个beta版，已经能无bug运行，排版还没定下来，调参不太方便。
体素模型比预期中快得多，比点云还快。
学习率递减还没写，这可能导致loss在下降中遇到瓶颈。
没有配准的程序，所以现在输入的数据是乱的。
'''

'更新日志09July\n这是第一个beta版，已经能无bug运行，排版还没定下来，调参不太方便。\n体素模型比预期中快得多，比点云还快。\n学习率递减还没写，这可能导致loss在下降中遇到瓶颈。\n没有配准的程序，所以现在输入的数据是乱的。\n'

In [3]:
'''
双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
通道1：原始牙齿的体素模型
通道2：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n通道1：原始牙齿的体素模型\n通道2：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [4]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [5]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh
from sklearn.model_selection import KFold

In [7]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 加载并归一化打磨前的初始模型
initial_mesh_path = 'stdTooth.obj'
initial_mesh = normalize_mesh(load_mesh(initial_mesh_path))
voxel_size = 32  # 体素网格尺寸
voxel_data_initial = mesh_to_voxel(initial_mesh, voxel_size)

class TeethDataset(Dataset):
    def __init__(self, initial_voxel, score_file_path, voxel_size=32):
        self.initial_voxel = initial_voxel
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        # 加载并归一化打磨后的模型
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data_after = mesh_to_voxel(mesh, self.voxel_size)
        # 双通道数据
        sample = np.stack([self.initial_voxel, voxel_data_after], axis=0)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor([score], dtype=torch.float32)  # 注意这里将 score 包装为列表


In [8]:
# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(voxel_data_initial, score_file_path, voxel_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [9]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(2, 32, kernel_size=3, stride=1, padding=1)  # 双通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
model = Teeth3DCNN()
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

In [10]:
'''
10epoch实验中每epoch loss不稳定，30epoch跑过两次结果都可以
泛化能力未知
'''

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 588.5508546327289
Epoch 2/30, Loss: 365.5397298586996
Epoch 3/30, Loss: 291.0398153505827
Epoch 4/30, Loss: 276.4801697981985
Epoch 5/30, Loss: 298.26150111148235
Epoch 6/30, Loss: 283.65872413233706
Epoch 7/30, Loss: 332.1481339304071
Epoch 8/30, Loss: 292.00204036110324
Epoch 9/30, Loss: 264.9424341101396
Epoch 10/30, Loss: 316.58242007305745
Epoch 11/30, Loss: 320.0903037221808
Epoch 12/30, Loss: 281.5353031158447
Epoch 13/30, Loss: 236.9160719545264
Epoch 14/30, Loss: 249.49736509825053
Epoch 15/30, Loss: 236.2648716976768
Epoch 16/30, Loss: 216.67354681617334
Epoch 17/30, Loss: 186.93058942336785
Epoch 18/30, Loss: 271.57961132651883
Epoch 19/30, Loss: 169.5839796317251
Epoch 20/30, Loss: 173.920517821061
Epoch 21/30, Loss: 176.01103152726827
Epoch 22/30, Loss: 140.79897753815902
Epoch 23/30, Loss: 102.32495814875553
Epoch 24/30, Loss: 124.94147323307239
Epoch 25/30, Loss: 86.84393999451085
Epoch 26/30, Loss: 98.66469418375115
Epoch 27/30, Loss: 80.22621018165036

In [11]:
'''
# K 折训练

kf = KFold(n_splits=5)
num_epochs = 10
for fold, (train_index, val_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}/{kf.n_splits}")
    
    # 重新初始化模型和优化器
    model = Teeth3DCNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 创建训练和验证数据子集
    train_subset = torch.utils.data.Subset(dataset, train_index)
    val_subset = torch.utils.data.Subset(dataset, val_index)
    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)
    
    # 训练模型
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # 验证模型
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")
'''

'\n# K 折训练\n\nkf = KFold(n_splits=5)\nnum_epochs = 10\nfor fold, (train_index, val_index) in enumerate(kf.split(dataset)):\n    print(f"Fold {fold + 1}/{kf.n_splits}")\n    \n    # 重新初始化模型和优化器\n    model = Teeth3DCNN()\n    criterion = nn.MSELoss()\n    optimizer = optim.Adam(model.parameters(), lr=0.001)\n    \n    # 创建训练和验证数据子集\n    train_subset = torch.utils.data.Subset(dataset, train_index)\n    val_subset = torch.utils.data.Subset(dataset, val_index)\n    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)\n    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)\n    \n    # 训练模型\n    for epoch in range(num_epochs):\n        model.train()\n        running_loss = 0.0\n        for inputs, targets in train_loader:\n            optimizer.zero_grad()\n            outputs = model(inputs)\n            loss = criterion(outputs, targets)\n            loss.backward()\n            optimizer.step()\n            running_loss += loss.item()\n        \n        # 验证模型

In [12]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 74.28167724609375
Tooth 2: Score = 70.59437561035156
Tooth 3: Score = 70.28372955322266
Tooth 4: Score = 70.97480010986328
Tooth 5: Score = 70.60059356689453
Tooth 6: Score = 65.2105484008789
Tooth 7: Score = 77.0385971069336
Tooth 8: Score = 79.59076690673828
Tooth 9: Score = 82.72244262695312
Tooth 10: Score = 75.48110961914062
Tooth 11: Score = 53.4840202331543
Tooth 12: Score = 66.3726806640625
Tooth 13: Score = 64.58563995361328
Tooth 14: Score = 69.4881362915039
Tooth 15: Score = 57.981590270996094
Tooth 16: Score = 47.8429069519043
Tooth 17: Score = 61.11915969848633
Tooth 18: Score = 62.88529968261719
Tooth 19: Score = 67.13316345214844
Tooth 20: Score = 64.68755340576172
Tooth 21: Score = 67.16334533691406
Tooth 22: Score = 66.7458267211914
Tooth 23: Score = 76.50544738769531
Tooth 24: Score = 72.79335021972656
Tooth 25: Score = 61.970863342285156
Tooth 26: Score = 60.85779571533203
Tooth 27: Score = 71.7269287109375
Tooth 28: Score = 67.46121978759766
Tooth 2

In [13]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 74.28167724609375 Err = -0.71832275390625
Tooth 2: Score = 70.59437561035156 Err = 2.5943756103515625
Tooth 3: Score = 70.28372955322266 Err = 0.28372955322265625
Tooth 4: Score = 70.97480010986328 Err = 1.9748001098632812
Tooth 5: Score = 70.60059356689453 Err = -5.399406433105469
Tooth 6: Score = 65.2105484008789 Err = 1.2105484008789062
Tooth 7: Score = 77.0385971069336 Err = 8.038597106933594
Tooth 8: Score = 79.59076690673828 Err = 6.590766906738281
Tooth 9: Score = 82.72244262695312 Err = 5.722442626953125
Tooth 10: Score = 75.48110961914062 Err = -1.518890380859375
Tooth 11: Score = 53.4840202331543 Err = -0.5159797668457031
Tooth 12: Score = 66.3726806640625 Err = 4.3726806640625
Tooth 13: Score = 64.58563995361328 Err = 5.585639953613281
Tooth 14: Score = 69.4881362915039 Err = 6.488136291503906
Tooth 15: Score = 57.981590270996094 Err = 0.9815902709960938
Tooth 16: Score = 47.8429069519043 Err = -0.15709304809570312
Tooth 17: Score = 61.11915969848633 Err = 1